# Hyperparameter Optimization

In [1]:
import numpy as np
from scipy.stats.distributions import expon, uniform, randint
from sklearn.model_selection import train_test_split, ParameterSampler
%matplotlib inline
import matplotlib.pyplot as plt

import wandb
from wandb.keras import WandbCallback

from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from keras.utils import to_categorical

Using TensorFlow backend.


Let's define a couple of helper functions:

In [2]:
def print_dict(d):
    for k, v in d.items():
        print('  {:>20}: {}'.format(k, v))
        
def print_header(s):
    divider = '=' * (len(s) + 4)
    print()
    print(divider)
    print('  {}  '.format(s))
    print(divider)

# Prepare Data

In [3]:
(X_train_valid, y_train_valid), (X_test, y_test) = cifar10.load_data()
X_train_valid = X_train_valid.astype('float32') / 255.
X_test = X_test.astype('float32') / 255.

y_train_valid = to_categorical(y_train_valid)
y_test = to_categorical(y_test)

X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_valid, y_train_valid, test_size=0.10)

In [4]:
print('Train shapes: x = {}, y = {}'.format(
    X_train.shape, y_train.shape))
print('Valid shapes: x = {}, y = {}'.format(
    X_valid.shape, y_valid.shape))
print('Test  shapes: x = {}, y = {}'.format(
    X_test.shape, y_test.shape))

Train shapes: x = (45000, 32, 32, 3), y = (45000, 10)
Valid shapes: x = (5000, 32, 32, 3), y = (5000, 10)
Test  shapes: x = (10000, 32, 32, 3), y = (10000, 10)


# Model Creation
Make a function which accepts a config object containing your hyperparameters and returns a compiled model.

In [5]:
def build_compile(config):
    model = Sequential()
    
    # first convolution / pooling set
    model.add(Conv2D(config.conv1_num_filters, 
                     config.conv_filter_size, 
                     activation=config.activation, 
                     padding='same',
                     input_shape=X_train.shape[1:]))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # second convolution / pooling set
    model.add(Conv2D(config.conv2_num_filters, 
                     config.conv_filter_size, 
                     padding='same',
                     activation=config.activation))
    model.add(Conv2D(config.conv3_num_filters, 
                     config.conv_filter_size, 
                     padding='same',
                     activation=config.activation))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # third convolution / pooling set
    model.add(Conv2D(config.conv4_num_filters, 
                     config.conv_filter_size, 
                     padding='same',
                     activation=config.activation))
    model.add(Conv2D(config.conv5_num_filters, 
                     config.conv_filter_size, 
                     padding='same',
                     activation=config.activation))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(config.dense1_size,
                    activation=config.activation))
    model.add(Dropout(config.dropout))
    model.add(Dense(10, activation='softmax'))

    model.compile(loss='categorical_crossentropy',
                  optimizer=Adam(lr=config.learn_rate),
                  metrics=['accuracy'])
    return model

# Hyperparameter Selection
Define the legal ranges for your hyperparameters and use `Sklearn`'s `ParameterSampler` to sample hyperparameters sets.

In [6]:
# powers of 2 tend to be good spreads
# learn_rate tends to be most important, need to get within 2x of optimal.
#     too high, and you step out (vaselate sp?) of your loss valley, too low and you learn too slowly
#     generally give a big spread of learn_rate, kitchen sink time
# How do we know which params?
# Also, batch_size and learn_rate correlated, batch up learn goes up
# Have to explore the space, very hard to manually guess correctly at the same level of efficiency
# Lots of trial and error
# TODO: how do you know which params are important?
hp_ranges = {
    'conv1_num_filters': [32, 64, 128],
    'conv2_num_filters': [32, 64, 128],
    'conv3_num_filters': [32, 64, 128],
    'conv4_num_filters': [32, 64, 128],
    'conv5_num_filters': [32, 64, 128],
    'dense1_size':       [32, 64, 128, 256, 512],
    'dropout':           uniform, # comes from sklearn above, random num gen?
    'learn_rate':        [0.1, 0.03, 0.001],
    'batch_size':        [8, 16, 32, 64, 128],
}

hp_sets = ParameterSampler(hp_ranges, n_iter=2, random_state=99)

In [7]:
for i, hp_set in enumerate(hp_sets):
    print()
    print("Hyperparameter Set {}:".format(i))
    print_dict(hp_set)


Hyperparameter Set 0:
            batch_size: 16
     conv1_num_filters: 64
     conv2_num_filters: 32
     conv3_num_filters: 64
     conv4_num_filters: 32
     conv5_num_filters: 128
           dense1_size: 512
               dropout: 0.8080499633648477
            learn_rate: 0.03

Hyperparameter Set 1:
            batch_size: 64
     conv1_num_filters: 64
     conv2_num_filters: 64
     conv3_num_filters: 32
     conv4_num_filters: 64
     conv5_num_filters: 128
           dense1_size: 256
               dropout: 0.7467671009942997
            learn_rate: 0.001


# Training

Static hyperparameters:

In [8]:
static_hyper_params = {
    'activation': 'relu',
    'conv_filter_size': 3,
    'num_epochs': 2,
}

Loop over `hp_sets`:

In [10]:
best_valid_acc = 0.0
best_hp_set = None
best_hp_ind = None

for hp_ind, hp_set in enumerate(hp_sets):
    # set up wandb
    print_header("Starting Training for Hyperparameter Set {}:".format(i))
    wandb.init()
    ## For short runs like this, wandb.monitor()
    # is just visual noise.  Reenable it for longer runs.
    # wandb.monitor()
    print_dict(hp_set)
   
    wandb.config.update(static_hyper_params, allow_val_change=True)
    wandb.config.update(hp_set, allow_val_change=True)

    # build model
    model = build_compile(wandb.config)
    print(model.summary())
    wandb.config.num_model_parameters = model.count_params()
    
    # train model
    history = model.fit(
        X_train, y_train,
        batch_size=wandb.config.batch_size,
        epochs=wandb.config.num_epochs,
        verbose=1,
        validation_data=(X_valid, y_valid),
        callbacks=[WandbCallback()]
    )
    
    # track best model so far
    valid_acc = history.history['val_acc'][-1]
    if valid_acc > best_valid_acc:
        best_valid_acc = valid_acc
        best_hp_set = hp_set
        best_hp_ind = hp_ind

wandb: ERROR: No W&B project configured.



  Starting Training for Hyperparameter Set 1:  
W&B Run: https://app.wandb.ai/mattlott/ztdl-lab30/runs/sc1gna3q
Call `%%wandb` in the cell containing your training loop to display live results.
            batch_size: 16
     conv1_num_filters: 64
     conv2_num_filters: 32
     conv3_num_filters: 64
     conv4_num_filters: 32
     conv5_num_filters: 128
           dense1_size: 512
               dropout: 0.8080499633648477
            learn_rate: 0.03
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 64)        1792      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 32)        18464     
_________________________________________________________________
conv2d_3 (Conv

# Retrain Best Model on Full train+valid Data

In [11]:
print_header("Best Hyperparams were set {} with valid accuracy {}".format(best_hp_ind, best_valid_acc))
print_dict(best_hp_set)

# Retrain model on combined training and validation data
wandb.config.update(best_hp_set)
model = build_compile(wandb.config)
history = model.fit(
    X_train_valid, y_train_valid,
    batch_size=wandb.config.batch_size,
    epochs=wandb.config.num_epochs,
    verbose=1,
    callbacks=[WandbCallback()]
)


  Best Hyperparams were set 1 with valid accuracy 0.5646  
            batch_size: 64
     conv1_num_filters: 64
     conv2_num_filters: 64
     conv3_num_filters: 32
     conv4_num_filters: 64
     conv5_num_filters: 128
           dense1_size: 256
               dropout: 0.7467671009942997
            learn_rate: 0.001
Epoch 1/2
50000/50000 [==============================] - 20s 392us/step - loss: 1.7780 - acc: 0.3401
Resuming run: https://app.wandb.ai/mattlott/ztdl-lab30/runs/7fqzobxq
Epoch 2/2
50000/50000 [==============================] - 19s 370us/step - loss: 1.3655 - acc: 0.5127


In [12]:
loss, acc = model.evaluate(X_test, y_test,
                           batch_size=wandb.config.batch_size)
print("Test loss: {}, test acc: {}".format(loss, acc))

10000/10000 [==============================] - 1s 128us/step
Test loss: 1.2283068012237548, test acc: 0.5685


### Inspect Results on WandB 
Go to https://app.wandb.ai/, then select your project name to see a summary of all your runs.

# Hyperparameter Gotchas
* It's easy to accidentally explode the size of your model.  In particular you get lots of parameters when:
  * You don't use much MaxPooling
  * You have a large first Dense layer after you Conv layers.
* As batch size goes up, learning rate can go up.  As batch size goes down, learning rate must go down.  Why?
  * More data leads to better gradient estimates

## Exercise 1:
* Create a function, `build_compile_ex1`, which can create a CNN with a variable number of convolutional and dense layers using the hyperparameter ranges below.
  * Remember that you'll need to special case the first conv layer to set `input_shape`.
  * The hyperparameter `num_convs_per_max_pool` chooses how many conv layers should pass between each max pooling layer. 
    * You'll probably find python's modulus division operator useful for this.  e.g.: `5 % 3 ==> 2; 6 % 3 ==> 0`
* Use the hyperparameter sets in `hp_sets_ex1` as your hyperparameter samples.
* The number of filters in each conv layer can be constant, the number of neurons in the dense layer should be constant.
* Include a `Dropout` layer after each `Dense` layer.
* Don't forget the `Flatten` layer before switching to `Dense`.

In [13]:
# Legal Hyperparameter Ranges
hp_ranges_ex1 = {
    'num_conv_filters':       [32, 64, 128],
    'num_conv_layers':        randint(2, 8),
    'num_convs_per_max_pool': randint(1, 3),
    'dense_size':             [32, 64, 128, 256, 512],
    'num_dense_layers':       randint(1, 3),
    'dropout':                uniform,
    'learn_rate':             [0.1, 0.03, 0.001],
    'batch_size':             [8, 16, 32, 64, 128],
}

hp_sets_ex1 = ParameterSampler(hp_ranges_ex1, n_iter=2, random_state=1234)

In [14]:
for i, hp_set in enumerate(hp_sets_ex1):
    print()
    print("Hyperparameter Set {}:".format(i))
    print_dict(hp_set)


Hyperparameter Set 0:
            batch_size: 64
            dense_size: 512
               dropout: 0.6121118946772708
            learn_rate: 0.03
      num_conv_filters: 64
       num_conv_layers: 3
  num_convs_per_max_pool: 1
      num_dense_layers: 1

Hyperparameter Set 1:
            batch_size: 64
            dense_size: 512
               dropout: 0.15881535318443651
            learn_rate: 0.001
      num_conv_filters: 128
       num_conv_layers: 2
  num_convs_per_max_pool: 1
      num_dense_layers: 1


Define your `build_compile_ex1` function in the next cell:

In [26]:
def build_compile_ex1(config):
    model = Sequential()

    ######### YOUR CODE HERE #########

    for i in range(config.num_conv_layers):
        if i is 0:
            model.add(Conv2D(config.num_conv_filters, 
                             config.conv_filter_size, 
                             padding='same',
                             activation=config.activation,
                             input_shape=X_train.shape[1:]))
        else:
            model.add(Conv2D(config.num_conv_filters, 
                             config.conv_filter_size, 
                             padding='same',
                             activation=config.activation))
#         if config.num_convs_per_max_pool % (i+1) is 0:
# From TA solution:
# Example of down sampling
        if not i % config.num_convs_per_max_pool:
            model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())

#     Key to back propagation and loss function
    for i in range(config.num_dense_layers):
        model.add(Dense(config.dense_size,
                        activation=config.activation))
        model.add(Dropout(config.dropout))
    
    model.add(Dense(10, activation='softmax'))

    model.compile(loss='categorical_crossentropy',
                  optimizer=Adam(lr=config.learn_rate),
                  metrics=['accuracy'])

    return model

In [27]:
static_hyper_params = {
    'activation': 'relu',
    'conv_filter_size': 3,
    'num_epochs': 2,
}

best_valid_acc = 0.0
best_hp_set = None
best_hp_ind = None

for hp_ind, hp_set in enumerate(hp_sets_ex1):
    # set up wandb
    print_header("Starting Training for Hyperparameter Set {}:".format(i))
    wandb.init()
    ## For short runs like this, wandb.monitor()
    # is just visual noise.  Reenable it for longer runs.
    # wandb.monitor()
    print_dict(hp_set)
   
    wandb.config.update(static_hyper_params, allow_val_change=True)
    wandb.config.update(hp_set, allow_val_change=True)

    # build model
    model = build_compile_ex1(wandb.config)
    print(model.summary())
    wandb.config.num_model_parameters = model.count_params()
    
    # train model 
    history = model.fit(
        X_train, y_train,
        batch_size=wandb.config.batch_size,
        epochs=wandb.config.num_epochs,
        verbose=1,
        validation_data=(X_valid, y_valid),
        callbacks=[WandbCallback()]
    )
    
    valid_acc = history.history['val_acc'][-1]
    if valid_acc > best_valid_acc:
        best_valid_acc = valid_acc
        best_hp_set = hp_set
        best_hp_ind = hp_ind


  Starting Training for Hyperparameter Set 1:  
W&B Run: https://app.wandb.ai/mattlott/ztdl-lab30/runs/vw7m42xv
Call `%%wandb` in the cell containing your training loop to display live results.
            batch_size: 64
            dense_size: 512
               dropout: 0.6121118946772708
            learn_rate: 0.03
      num_conv_filters: 64
       num_conv_layers: 3
  num_convs_per_max_pool: 1
      num_dense_layers: 1
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_29 (Conv2D)           (None, 32, 32, 64)        1792      
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 16, 16, 64)        36928     
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 8, 8, 6

In [28]:
print_header("Best Hyperparams were set {} with valid accuracy {}".format(best_hp_ind, best_valid_acc))
print_dict(best_hp_set)

# Retrain model on combined training and validation data
wandb.config.update(best_hp_set)
model = build_compile_ex1(wandb.config)
history = model.fit(
    X_train_valid, y_train_valid,
    batch_size=wandb.config.batch_size,
    epochs=wandb.config.num_epochs,
    verbose=1,
    callbacks=[WandbCallback()]
)


  Best Hyperparams were set 1 with valid accuracy 0.6568  
            batch_size: 64
            dense_size: 512
               dropout: 0.15881535318443651
            learn_rate: 0.001
      num_conv_filters: 128
       num_conv_layers: 2
  num_convs_per_max_pool: 1
      num_dense_layers: 1
Epoch 1/2
50000/50000 [==============================] - 19s 374us/step - loss: 1.3879 - acc: 0.5013
Resuming run: https://app.wandb.ai/mattlott/ztdl-lab30/runs/9w3ibjnm
Epoch 2/2
50000/50000 [==============================] - 17s 349us/step - loss: 0.9960 - acc: 0.6495


In [29]:
loss, acc = model.evaluate(X_test, y_test, batch_size=wandb.config.batch_size)
print("Test loss: {}, test acc: {}".format(loss, acc))

10000/10000 [==============================] - 2s 170us/step
Test loss: 0.9220220293045044, test acc: 0.6816


## Exercise 2
* In practice, you don't conduct a hyperparameter search by wrapping many training runs in a for loop on a single machine.  
* Instead, you want to have a single machine which selects the hyperparameter sets, then sends them off to worker nodes which actually conduct the training.
* Multi-node training isn't hard to do, but it's out of scope for this 1-week class; too many IT hurdles.  In this exercise, though, we'll refactor our existing code to more closely approximate a real training setup.

### Instructions
* Refactor your existing code into a script rather than a notebook.
* The script should accept a series of keyword arguments containing all the hyperparameter values for a single run.  Check out the `argparse` python module.
* It should then combine these arguments into a Python dict representing a single hyperparameter set like the `hp_set` variable above.
* The script should then update the wandb.config object with the values from the input hyperparameter set and train a model using those values.  You don't need to save the final result anywhere, the `WandbCallback()` will take care of that for you.

In [ ]:
# Too much devops, left as homework

## Exercise 3
*  Create a large number of hyperparameter sets.
*  For each hyperparameter set, print out the model summary and study the number of parameters that are produced.  Try to get a sense for what configurations produce large parameter counts.
*  If you have time, train models based on some of these hyperparameter sets and see which produce good results and which don't.

In [31]:
# Num parameters = overfit more a problem, need more data.  consumes more GPU memory => slower
hp_sets_ex3 = ParameterSampler(hp_ranges_ex1, n_iter=100, random_state=1234)
len(hp_sets_ex3)

# for hp_ind, hp_set in enumerate(hp_sets_ex3):
#     # set up wandb
#     print_header("Starting Training for Hyperparameter Set {}:".format(i))
#     wandb.init()
#     ## For short runs like this, wandb.monitor()
#     # is just visual noise.  Reenable it for longer runs.
#     # wandb.monitor()
#     print_dict(hp_set)
   
#     wandb.config.update(static_hyper_params, allow_val_change=True)
#     wandb.config.update(hp_set, allow_val_change=True)

#     # build model
#     model = build_compile_ex1(wandb.config)
#     print(model.summary())
#     wandb.config.num_model_parameters = model.count_params()
    
#     # train model 
#     history = model.fit(
#         X_train, y_train,
#         batch_size=wandb.config.batch_size,
#         epochs=wandb.config.num_epochs,
#         verbose=1,
#         validation_data=(X_valid, y_valid),
#         callbacks=[WandbCallback()]
#     )
    
#     valid_acc = history.history['val_acc'][-1]
#     if valid_acc > best_valid_acc:
#         best_valid_acc = valid_acc
#         best_hp_set = hp_set
#         best_hp_ind = hp_ind

100